In [1]:
! head ./data/ml-1m/ratings.dat

1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368


In [ ]:
import numpy as np
import random
from tqdm import tqdm
from rank_metrics import ndcg_at_k
from sklearn.metrics import roc_auc_score

In [ ]:
raw_data = np.loadtxt('./data/ml-1m/ratings.dat', delimiter='::')

In [ ]:
class TriplesDataset(object):
    
    def __init__(self, raw_data, threshold_user = 20, threshold_item = 5, rnd_seed = 42):
        self.threshold_user = threshold_user
        self.threshold_item = threshold_item
        
        self.N_ITEMS = int(max(raw_data[:, 1]))
        
        # fix randomness
        self.seed = rnd_seed
        np.random.seed(rnd_seed)
        random.seed(rnd_seed)
        
        # calculate user/item counts
        u_cnt = {}
        i_cnt = {}
        for row in tqdm(raw_data, desc='Calculate counts', leave=False):
            u, i, r, t = row.astype(int)
            u_cnt[u] = u_cnt.get(u, 0) + 1
            i_cnt[i] = i_cnt.get(i, 0) + 1

        # make blacklists
        u_blacklist = set([k for k, v in u_cnt.items() if v < threshold_user])
        i_blacklist = set([k for k, v in i_cnt.items() if v < threshold_item])
        
        # assemble self.data like {user:[(item, rating),...], ...}
        self.data = {}
        self.used = {}
        for row in tqdm(raw_data, desc='Assemble .data', leave=False):
            u, i, r, t = list(map(int, row))
            # filtering rare
            if u in u_blacklist or i in i_blacklist:
                continue
            self.data[u] = self.data.get(u, []) + [(i, r)]
            self.used[u] = self.used.get(u, []) + [i]
        self.data_keys = list(self.data.keys())
        self.train_pool = None
    
    
    def train_test_split(self, n_train=None, n_test=None):
        assert n_train is None or n_test is None, 'both n_train and n_test cannot be set'
        #assert not (n_train is None and n_test is None), 'both n_train and n_test cannot be None'
        assert n_test is None, 'n_test not supported for now'
        
        # .train : {user: {rating: [items, ...], ...}}
        self.train = {}
            
        # .test : {user: [(item, rating), ..]} [sorted by rating]
        self.test = {}
            
        for u in tqdm(self.data, desc='Split users', leave=False):
            rnd_inds = set(np.random.choice(len(self.data[u]), n_train or n_test, False))
            for n, pair in enumerate(self.data[u]):
                if n in rnd_inds:
                    i, r = pair
                    user_dict = self.train.get(u, {})
                    rating_list = user_dict.get(r, [])
                    rating_list.append(i)
                    user_dict[r] = rating_list
                    self.train[u] = user_dict
                else:
                    self.test[u] = self.test.get(u, []) + [pair]
            if len(self.train[u].keys())==1:
                print('No rating diversity in train for user {}, do swap!'.format(u))
                the_only_rating = list(self.train[u].keys())[0]
                for n, (i, r) in enumerate(self.test[u]):
                    if r!=the_only_rating:
                        self.train[u][r] = [i]
                        extracted_i = self.train[u][the_only_rating][0]
                        self.train[u][the_only_rating] = self.train[u][the_only_rating][1:]
                        del self.test[u][n]
                        self.test[u] = self.test[u] + [(extracted_i, the_only_rating)]
                        break
            self.test[u] = sorted(self.test[u], key=lambda x: x[1], reverse=True)
                    
    def sample_train_triple(self):
        user = random.choice(self.data_keys)
        stats = self.train[user]
        stats_keys = list(stats.keys())
        assert len(stats_keys) > 1, 'user {} has only 1 rating!'.format(user)
          
        # custom sampler from 0 rating -- uses non-labeled data (as implicit feedback)
        def sampler(rating, stats, used):
            if rating > 0:
                return random.choice(stats[rating])
            while True:
                rnd = random.randint(0, self.N_ITEMS - 1)
                if rnd not in used:
                    return rnd
                
        left_rating, right_rating = random.sample(stats_keys + [0], 2)
        left_value = sampler(left_rating, stats, self.used[user])
        right_value = sampler(right_rating, stats, self.used[user])
        y = (left_rating > right_rating)*2 - 1
        return (user, left_value, right_value, y)
    
    
    def sample_train_batch(self,n_samples=256):
        retval = np.zeros((n_samples, 4)).astype(np.int32)
        for i in range(n_samples):
            retval[i] = self.sample_train_triple()
        return {
            'users': retval[:, 0], 
            'left_items': retval[:, 1],
            'right_items': retval[:, 2],
            'y': retval[:, 3].astype(np.float32),
        }

In [ ]:
ds = TriplesDataset(raw_data, threshold_user=30, rnd_seed=42)
ds.train_test_split(n_train=20)

# define model

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [ ]:
tf.__version__

In [ ]:
N_USERS = int(max(raw_data[:, 0])) + 1
N_ITEMS = int(max(raw_data[:, 1])) + 1
N_HIDDEN = 10

In [ ]:
# placeholders
user_ids  = tf.placeholder(tf.int32, (None,), name='user_ids')
left_ids  = tf.placeholder(tf.int32, (None,), name='left_ids')
right_ids = tf.placeholder(tf.int32, (None,), name='right_ids')
target_y  = tf.placeholder(tf.float32, (None,), name='target_y')


# main parameters
user_latents = tf.Variable(tf.random_uniform(shape=(N_USERS, N_HIDDEN)), trainable=True, name='user_latents')
item_latents = tf.Variable(tf.random_uniform(shape=(N_ITEMS, N_HIDDEN)), trainable=True, name='item_latents')

## define batch processing

# get embeddings for batch
embedding_user  = tf.nn.embedding_lookup(user_latents, user_ids, name='embedding_user')
embedding_left  = tf.nn.embedding_lookup(item_latents, left_ids, name='embedding_left')
embedding_right = tf.nn.embedding_lookup(item_latents, right_ids, name='embedding_right')

# raw margins for primal ranking loss
embedding_diff = embedding_left - embedding_right

# shape: [n_batch, ]
embedding_margins = tf.reduce_sum(tf.mul(embedding_user, embedding_diff), axis=1, name='embedding_margins')

In [ ]:
# losses
def mean_logloss(raw_margins, target_values, tag):
    # -y*f(x)
    myfx = -1*tf.mul(target_values, raw_margins)
    elementwise_logloss = tf.minimum(tf.log(1 + tf.exp(myfx)), 100, name='elwise_' + tag)
    mean_loss = tf.reduce_mean(elementwise_logloss, name='mean_' + tag)
    return mean_loss

embedding_loss = mean_logloss(embedding_margins, target_y, 'embedding_loss')
target = embedding_loss

In [ ]:
trainer_2 = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(target)
trainer_3 = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(target)
trainer_4 = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(target)

In [ ]:
init_vars = tf.global_variables_initializer()
sess.run(init_vars)

In [ ]:
losses = []
batch_size = 1024
for n_batches, cur_optim in [(3000, trainer_2)]:
    for i in tqdm(range(n_batches)):
        batch = ds.sample_train_batch(n_samples=batch_size)
        fd = {
            user_ids:  batch['users'], 
            left_ids:  batch['left_items'],
            right_ids: batch['right_items'],
            target_y:  batch['y'],
        }
        el, _ = sess.run([embedding_loss, cur_optim], feed_dict=fd)
        losses.append((el, 0.5))
        if i%1000==0:
            user_norm = np.linalg.norm(user_latents.eval())
            item_norm = np.linalg.norm(item_latents.eval())
            print('Weight norms, users: {}, items: {}'.format(user_norm, item_norm))

In [ ]:
%pylab inline
import random

In [ ]:
plot([x[0] for x in losses], c='b', label='embedding_loss', alpha=0.5)
plot([x[1] for x in losses], c='r', label='net_loss', alpha=0.5)
grid()
legend()
xlabel('n_batches')
ylabel('logloss')

In [ ]:
batch = ds.sample_train_batch(n_samples=1)
print(batch)

In [ ]:
fd = {
    user_ids:  batch['users'], 
    left_ids:  batch['left_items'],
    right_ids: batch['right_items'],
    target_y:  np.array([1]),
}
    
u, l, r, d, m, t = sess.run([embedding_user, embedding_left, embedding_right, embedding_diff, embedding_margins, target], fd)

In [ ]:
ndcg_vals = []
auc_vals = []
for u in tqdm(ds.data_keys, desc='Prediction', leave=True):
    fd = {
            user_ids:  (np.ones(N_ITEMS)*u).astype(np.int32), 
            left_ids:  np.array(range(N_ITEMS)).astype(np.int32),
        }
    response = sess.run(embedding_left, feed_dict=fd)[:, 0]

    
    
    # make response
    INF = 10e+5
    response[0] = -INF
    for rating, items in ds.train[u].items():
        for i in items:
            response[i] = -INF

    # make relevances
    relevances = np.zeros(N_ITEMS)
    for i, r in ds.test[u]:
        relevances[i] = r
    
    auc_vals.append(roc_auc_score(relevances > 0, response))
    
    sorted_ids = np.argsort(-response)
    relevances = relevances[sorted_ids]
    
    # calc score
    ndcg_vals.append(ndcg_at_k(relevances, 10))

In [ ]:
np.mean(ndcg_vals)

In [ ]:
np.mean(auc_vals)

In [ ]:
plot(ndcg_vals)

In [ ]:
ndcg_at_k(relevances, 10, method=1)